In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Geospatial Analysis with Places Insights Sample Data

### Overall Goal

This notebook serves as a technical introduction for developers and data analysts who have subscribed to the [Places Insights Sample Datasets](https://developers.google.com/maps/documentation/placesinsights/cloud-setup#sample_data).

Its primary purpose is to demonstrate how to query, aggregate, and visualize Google Maps Platform [Places Insights](https://developers.google.com/maps/documentation/placesinsights) data within a **BigQuery** environment. By running this notebook, you will learn how to transition from raw dataset subscriptions to actionable geospatial insights using Standard SQL and Python.

### Key Technologies Used

*   **[Places Insights](https://developers.google.com/maps/documentation/placesinsights):** A BigQuery dataset providing aggregated counts and attributes for Points of Interest (POIs).
*   **[BigQuery](https://cloud.google.com/bigquery):** Used to execute Standard SQL and Geospatial functions (such as `ST_DWITHIN` and `ST_GEOGPOINT`) on the dataset.
*   **[H3 (Hierarchical Geospatial Indexing)](https://h3geo.org/):** A hexagonal grid system used by the Places Insights SQL functions to normalize spatial data.
*   **[Google Maps 2D Tiles](https://developers.google.com/maps/documentation/tile/2d-tiles-overview):** Provides the high-resolution roadmap imagery used for the visualization layer.
*   **Python Libraries:**
    * **[Folium](https://python-visualization.github.io/folium/latest/)** for map rendering.
    * **[GeoPandas](https://geopandas.org/)** & **[Shapely](https://shapely.readthedocs.io/)** for processing coordinate reference systems and geometry objects.

**See [Google Maps Platform Pricing](https://mapsplatform.google.com/intl/en_uk/pricing/) and [BigQuery Pricing](https://cloud.google.com/bigquery/pricing) for costs associated with running this notebook.**

### The Step-by-Step Workflow

1.  **Configuration:** The notebook initializes the environment based on your selection of a Sample City (e.g., "New York City", "Tokyo", "London"). It automatically maps your selection to the correct **Analytics Hub Dataset ID** and geographic coordinates.

2.  **Direct Query Analysis (Radius Search):** We demonstrate how to count places that match specific criteria, such as `primary_type`, `business_status`, and boolean attributes like `allows_dogs`, within a set radius. We utilize the `WITH AGGREGATION_THRESHOLD` clause to compare amenity density across multiple neighborhoods programmatically.

3.  **H3 Density Analysis (Grid Search):** We utilize the predefined `PLACES_COUNT_PER_H3` SQL function to retrieve a normalized grid of place counts. This allows us to visualize macro-level commercial density across the entire city without manually defining boundaries.

4.  **Visualization:** We render the query results on interactive maps using **Folium** and **Google Maps 2D Tiles**:
    *   **Marker Map:** Visualizes the "Direct Query" results, identifying hotspots based on amenity concentration.
    *   **Choropleth Map:** Overlays the H3 hexagonal grid to visualize the "Function Query" density results.

### **How to Use This Notebook**

1.  **Prerequisites:**
    *   A Google Cloud Project with the **BigQuery API** and **Map Tiles API** enabled.
    *   A subscription to at least one of the [Places Insights Sample Datasets](https://developers.google.com/maps/documentation/placesinsights/cloud-setup#sample_data) via Analytics Hub.

2.  **Set Up Secrets:** Configure the following keys in the Colab "Secrets" tab (the **key icon** on the left menu):
    *   `GCP_PROJECT_ID`: Your Google Cloud Project ID.
    *   `GMP_API_KEY`: Your Google Maps Platform API key.

3.  **Run the Cells:** Run the cells in sequence to authenticate, execute the BigQuery jobs, and render the visualizations.

In [ ]:
# @title 1. Setup & Authentication
# @markdown Authenticate to Google Cloud, retrieve secrets, and initialize the BigQuery client.

import sys
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
from google.colab import userdata, auth
from google.cloud import bigquery

# 1. Retrieve Secrets
try:
    GCP_PROJECT_ID = userdata.get('GCP_PROJECT_ID')
    GMP_API_KEY = userdata.get('GMP_API_KEY')
    print(f"✅ Secrets retrieved for project: {GCP_PROJECT_ID}")
except userdata.SecretNotFoundError as e:
    raise ValueError("Missing Secrets! Please add 'GCP_PROJECT_ID' and 'GMP_API_KEY' to Colab Secrets.") from e

# 2. Authenticate User
# This will open a pop-up to login with your Google Account
auth.authenticate_user(project_id=GCP_PROJECT_ID)
print("✅ User Authenticated.")

# 3. Initialize BigQuery Client
client = bigquery.Client(project=GCP_PROJECT_ID)
print("✅ BigQuery Client Initialized.")

In [ ]:
# @title 2. Configuration
# @markdown Select a Sample City to demo. This configures the dataset target and map center.
# @markdown
# @markdown **Important:** You must be subscribed to the [Sample Dataset](https://developers.google.com/maps/documentation/placesinsights/cloud-setup#sample_data) in your GCP project.
# @markdown
# @markdown If you renamed the dataset during the Analytics Hub subscription process, enter the full table name (e.g., `my_project.my_dataset.places_sample`) in the **Custom Table Name** field below. Leave it blank to use the default.

SAMPLE_LOCATION = "London, UK" # @param ["Sydney, Australia", "Sao Paulo, Brazil", "Toronto, Canada", "Paris, France", "Berlin, Germany", "Mumbai, India", "Jakarta, Indonesia", "Rome, Italy", "Tokyo, Japan", "Mexico City, Mexico", "Madrid, Spain", "Zurich, Switzerland", "London, UK", "New York City, USA"]
CUSTOM_TABLE_NAME = "" # @param {type:"string"}

# Configuration Dictionary mapping selection to Dataset and Center Coordinates
# Note: These table names assume the default naming convention from Analytics Hub.
config_map = {
    "Sydney, Australia":    {"code": "au", "center": (-33.8688, 151.2093)},
    "Sao Paulo, Brazil":    {"code": "br", "center": (-23.5505, -46.6333)},
    "Toronto, Canada":      {"code": "ca", "center": (43.65107, -79.347015)},
    "Paris, France":        {"code": "fr", "center": (48.8566, 2.3522)},
    "Berlin, Germany":      {"code": "de", "center": (52.5200, 13.4050)},
    "Mumbai, India":        {"code": "in", "center": (19.0760, 72.8777)},
    "Jakarta, Indonesia":   {"code": "id", "center": (-6.2088, 106.8456)},
    "Rome, Italy":          {"code": "it", "center": (41.9028, 12.4964)},
    "Tokyo, Japan":         {"code": "jp", "center": (35.6895, 139.6917)},
    "Mexico City, Mexico":  {"code": "mx", "center": (19.4326, -99.1332)},
    "Madrid, Spain":        {"code": "es", "center": (40.4168, -3.7038)},
    "Zurich, Switzerland":  {"code": "ch", "center": (47.3769, 8.5417)},
    "London, UK":           {"code": "gb", "center": (51.5072, -0.1276)},
    "New York City, USA":   {"code": "us", "center": (40.7580, -73.9855)},
}

selected_config = config_map[SAMPLE_LOCATION]
CITY_CENTER_LAT, CITY_CENTER_LNG = selected_config['center']

# Determine Table Name
if CUSTOM_TABLE_NAME and CUSTOM_TABLE_NAME.strip():
    DATASET_TABLE = CUSTOM_TABLE_NAME.strip()
    print(f"⚠️ Using Custom Table Name: {DATASET_TABLE}")
else:
    # Default construction
    DATASET_TABLE = f"places_insights___{selected_config['code']}___sample.places_sample"
    print(f"🎯 Target Table (Default): {DATASET_TABLE}")

print(f"🌍 Selected Location: {SAMPLE_LOCATION}")
print(f"📍 Center Point: {CITY_CENTER_LAT}, {CITY_CENTER_LNG}")

In [ ]:
# @title 3. Direct Query: Analyzing "Al Fresco & Dog-Friendly" Hubs
# @markdown The following two cells are an example of [querying the sample data directly](https://developers.google.com/maps/documentation/placesinsights/queries), and visualizing the results on a map.
# @markdown
# @markdown We calculate the count of Operational Restaurants that are either Dog-Friendly OR have Outdoor Seating.
# @markdown We use a `TEMP TABLE` and a single SQL `JOIN` to analyze all 5 locations simultaneously.

# --- 1. Generate Candidate Locations (Python) ---
def get_offset_point(lat, lng, lat_offset, lng_offset):
    return lat + lat_offset, lng + lng_offset

offsets = [
    ("City Center", 0, 0),
    ("North District", 0.015, 0),
    ("South District", -0.015, 0),
    ("East District", 0, 0.02),
    ("West District", 0, -0.02)
]

# Save metadata for Python merging later
location_metadata = []
values_list = []

for name, lat_off, lng_off in offsets:
    lat, lng = get_offset_point(CITY_CENTER_LAT, CITY_CENTER_LNG, lat_off, lng_off)
    location_metadata.append({"name": name, "lat": lat, "lng": lng})
    # ST_GEOGPOINT is (Longitude, Latitude)
    values_list.append(f"('{name}', ST_GEOGPOINT({lng}, {lat}))")

values_sql = ",\n    ".join(values_list)

# --- 2. SQL Script ---
# We use a multi-statement script to Create, Insert, then Select.
query = f"""
CREATE TEMP TABLE temp_analysis_locations (
    name STRING,
    center GEOGRAPHY
);

INSERT INTO temp_analysis_locations (name, center)
VALUES
    {values_sql};

SELECT WITH AGGREGATION_THRESHOLD
    loc.name,
    COUNT(*) as count
FROM `{DATASET_TABLE}` places
JOIN temp_analysis_locations loc
  ON ST_DWITHIN(places.point, loc.center, 1000)
WHERE
  'restaurant' IN UNNEST(places.types)
  AND places.business_status = 'OPERATIONAL'
  AND (places.allows_dogs = TRUE OR places.outdoor_seating = TRUE)
GROUP BY loc.name
ORDER BY count DESC;
"""

print("🚀 Running BigQuery Analysis...")

try:
    df_results = client.query(query).to_dataframe()
except Exception as e:
    print(f"❌ Query Failed: {e}")
    df_results = pd.DataFrame(columns=['name', 'count'])

# --- 3. Merge & Score (Python) ---
if not df_results.empty:
    # Convert metadata to DataFrame
    df_locs = pd.DataFrame(location_metadata)

    # Merge BigQuery results with our Python Coordinates
    df_scored = pd.merge(df_locs, df_results, on='name', how='left')

    # Fill NaNs with 0 (for areas with <5 results that BigQuery omitted)
    df_scored['count'] = df_scored['count'].fillna(0).astype(int)

    # Sort for scoring
    df_scored = df_scored.sort_values('count', ascending=False).reset_index(drop=True)

    # Apply Relative Coloring
    num_results = len(df_scored)
    categories = []
    colors = []

    for i in range(num_results):
        # Determine category based on rank
        if df_scored.iloc[i]['count'] == 0:
            cat, col = "Low/No Activity", "gray"
        elif i == 0:
            cat, col = "🔥 Top Hotspot", "red"
        elif i == num_results - 1:
            cat, col = "🧊 Lowest Activity", "blue"
        else:
            cat, col = "Active Area", "orange"

        categories.append(cat)
        colors.append(col)

    df_scored['category'] = categories
    df_scored['color'] = colors

    print("\n✅ Analysis Complete.")
    display(df_scored)
else:
    print("⚠️ No results found. It is possible all counts were below the privacy threshold (5).")
    df_scored = pd.DataFrame()

In [ ]:
# @title 4. Visualization
# @markdown We map the locations from the previous cell. **Red** = Highest concentration, **Blue** = Lowest concentration.

import folium
from folium import plugins

# 1. Validation
if 'df_scored' not in locals() or df_scored.empty:
    raise ValueError("No data found! Please run Phase 3 successfully first.")
if not GMP_API_KEY:
    raise ValueError("GMP_API_KEY is missing. Please set it in Phase 1.")

# 2. Create Google Maps Session (Required for 2D Tiles)
print("🗺️ Initializing Google Maps Session...")
session_url = f"https://tile.googleapis.com/v1/createSession?key={GMP_API_KEY}"
headers = {"Content-Type": "application/json"}
payload = {
    "mapType": "roadmap",
    "language": "en-US",
    "region": "US"
}

try:
    response = requests.post(session_url, json=payload, headers=headers)
    response.raise_for_status()
    session_token = response.json().get("session")
    print("✅ Session Token acquired.")
except Exception as e:
    raise RuntimeError(f"Failed to initialize Google Maps session: {e}")

# 3. Fetch Dynamic Attribution
min_lat, max_lat = df_scored['lat'].min(), df_scored['lat'].max()
min_lng, max_lng = df_scored['lng'].min(), df_scored['lng'].max()
zoom_level = 13

viewport_url = (
    f"https://tile.googleapis.com/tile/v1/viewport?key={GMP_API_KEY}"
    f"&session={session_token}"
    f"&zoom={zoom_level}"
    f"&north={max_lat}&south={min_lat}"
    f"&west={min_lng}&east={max_lng}"
)

try:
    vp_response = requests.get(viewport_url)
    vp_response.raise_for_status()
    viewport_data = vp_response.json()
    google_attribution = viewport_data.get('copyright', '© Google')
except Exception as e:
    print(f"⚠️ Warning: Could not fetch attribution ({e}). Defaulting to standard.")
    google_attribution = "Map data © Google"

# 4. Initialize Map
tiles_url = f"https://tile.googleapis.com/v1/2dtiles/{{z}}/{{x}}/{{y}}?session={session_token}&key={GMP_API_KEY}"

m = folium.Map(
    location=[CITY_CENTER_LAT, CITY_CENTER_LNG],
    zoom_start=zoom_level,
    tiles=tiles_url,
    attr=google_attribution,
    name="Google Maps"
)

# 5. Add Markers
for _, row in df_scored.iterrows():
    # Icon Selection
    if row['color'] == 'red':
        icon_type = 'fire'
    elif row['color'] == 'blue':
        icon_type = 'minus-sign'
    else:
        icon_type = 'cutlery'

    # Rich HTML Popup
    popup_html = f"""
    <div style="font-family: sans-serif; width: 200px;">
        <h4 style="margin-bottom:5px;">{row['name']}</h4>
        <span style="color:{row['color']}; font-weight:bold; font-size: 1.1em;">{row['category']}</span>
        <hr>
        <b>Outdoor/Dog-Friendly Dining:</b><br>
        <span style="font-size: 2em; font-weight: bold;">{row['count']}</span> places
    </div>
    """

    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=f"{row['name']} ({row['count']})",
        icon=folium.Icon(color=row['color'], icon=icon_type)
    ).add_to(m)

# Add Legend
legend_html = '''
     <div style="position: fixed; bottom: 50px; left: 50px; width: 160px; height: 100px;
     border:2px solid grey; z-index:9999; font-size:14px; background-color:white; opacity:0.9; padding: 10px;">
     <b>Dining Activity</b><br>
     &nbsp; <i class="fa fa-fire" style="color:red"></i> &nbsp; Top Hotspot<br>
     &nbsp; <i class="fa fa-cutlery" style="color:orange"></i> &nbsp; Moderate<br>
     &nbsp; <i class="fa fa-minus-circle" style="color:blue"></i> &nbsp; Quietest<br>
     </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

display(m)

In [ ]:
# @title 5a. Function Query: Fetch H3 Density Data
# @markdown The following two cells are an example of [querying the sample data using a Function](https://developers.google.com/maps/documentation/placesinsights/place-count-functions/function-queries), and visualizing the results on a map.
# @markdown
# @markdown We use the [PLACES_COUNT_PER_H3](https://developers.google.com/maps/documentation/placesinsights/place-count-functions/places-count-per-h3) function to retrieve aggregation data.
# @markdown **Note:** We calculate density for a 5km radius with H3 resolution 8.

import json
from shapely.geometry import shape

# 1. robustly determine Function Name from Table Name
# We assume the structure is `project.dataset.table` or `dataset.table`.
# We need to strip the `.table` part and append `.PLACES_COUNT_PER_H3`.
if "." in DATASET_TABLE:
    dataset_path = DATASET_TABLE.rsplit('.', 1)[0]
    function_table_name = f"{dataset_path}.PLACES_COUNT_PER_H3"
else:
    # Fallback if provided string has no dots (unlikely for full paths)
    function_table_name = "PLACES_COUNT_PER_H3"

print(f"🎯 Function Target: {function_table_name}")

# 2. Define Function Query
# Fixed: h3_resolution set to 8 (Max supported value)
function_query = f"""
SELECT
    h3_cell_index,
    count,
    ST_ASGEOJSON(geography) as geometry_json
FROM `{function_table_name}`(
    JSON_OBJECT(
        'geography', ST_GEOGPOINT({CITY_CENTER_LNG}, {CITY_CENTER_LAT}),
        'geography_radius', 5000,
        'types', ['restaurant', 'store', 'point_of_interest'],
        'business_status', ['OPERATIONAL'],
        'h3_resolution', 8
    )
)
"""

print("🚀 Running H3 Function Query...")
try:
    df_h3 = client.query(function_query).to_dataframe()
    print(f"✅ Data Retrieved: {len(df_h3)} hexagonal cells.")
except Exception as e:
    print(f"❌ Query Failed: {e}")
    print("👉 Hint: Ensure you have subscribed to the Sample Dataset for this specific country in Analytics Hub.")
    df_h3 = pd.DataFrame()

# 3. Process Data for Mapping
if not df_h3.empty:
    print("⚙️ Processing Geometries...")
    # Convert GeoJSON string (from BigQuery) to Shapely Geometry objects
    df_h3['geometry'] = df_h3['geometry_json'].apply(lambda x: shape(json.loads(x)))

    # Create GeoDataFrame
    gdf_h3 = gpd.GeoDataFrame(df_h3, geometry='geometry')

    # CRITICAL FIX: Set the Coordinate Reference System (CRS) to WGS84 (EPSG:4326)
    # This tells GeoPandas/Folium that these are real-world Lat/Lon coordinates.
    gdf_h3.set_crs(epsg=4326, inplace=True)

    print("✅ GeoDataFrame Ready.")
else:
    gdf_h3 = None
    print("⚠️ No data to process.")

In [ ]:
# @title 5b. Function Query: Render Heatmap
# @markdown We visualize the density data as a Choropleth layer on the Google Maps basemap.

# 1. Validation
if 'gdf_h3' not in locals() or gdf_h3 is None or gdf_h3.empty:
    raise ValueError("No H3 data found! Please run Cell 5a successfully first.")

# 2. Map Setup
# We check if the tiles_url exists from Cell 4, otherwise we assume standard tiles (or re-auth if needed)
if 'tiles_url' not in locals():
    print("⚠️ Google Tiles session not found from Cell 4. Falling back to standard OpenStreetMap.")
    tiles_url = "OpenStreetMap"
    attr = "OpenStreetMap"
else:
    attr = google_attribution

m_hex = folium.Map(
    location=[CITY_CENTER_LAT, CITY_CENTER_LNG],
    zoom_start=12, # Zoomed out slightly for the larger 5km radius
    tiles=tiles_url,
    attr=attr,
    name="Google Maps"
)

# 3. Add Choropleth Layer
# We use the H3 density count to color the hexagons
folium.Choropleth(
    geo_data=gdf_h3,
    data=df_h3,
    columns=['h3_cell_index', 'count'],
    key_on='feature.properties.h3_cell_index',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.1,
    legend_name='Place Density (Count per H3 Cell)'
).add_to(m_hex)

print("🗺️ Rendering Heatmap...")
display(m_hex)